In [2]:
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import json
from tqdm import tqdm
import os

In [ ]:
HF_TOKEN = "your_huggingface_token_here"  # Replace with your Hugging Face token
MODEL_ID = "Viet-Mistral/Vistral-7B-Chat"
INPUT_PATH = "/kaggle/input/vlsp-dataset/public_test.txt"
OUTPUT_PATH = "/kaggle/working/vistral_predictions.txt"
MAX_NEW_TOKENS = 64
BATCH_SIZE = 4

In [ ]:
!pip install --upgrade bitsandbytes
!pip install --upgrade accelerate
!pip install --upgrade transformers


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import BitsAndBytesConfig
!pip install -U bitsandbytes
# ====== Auth + Load Model ======
print("Đăng nhập Hugging Face...")
login(token=HF_TOKEN)

# ====== Quantization config ======
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # hoặc load_in_4bit=True nếu muốn 4-bit
    bnb_4bit_compute_dtype="float16",  # dtype tính toán
    bnb_4bit_use_double_quant=True
)

# ====== Tokenizer ======
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_auth_token=HF_TOKEN)

# ====== Load model với quantization ======
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",        # auto phân phối GPU/CPU
    offload_folder="/tmp"     # offload layer chưa dùng sang CPU
)

# ====== Pipeline ======
chat_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=64,

)

Đăng nhập Hugging Face...


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:1001: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [7]:
# ====== Load Input Data ======
print(f"Đọc dữ liệu từ: {INPUT_PATH}")
with open(INPUT_PATH, 'r', encoding='utf-8') as f:
    data = [json.loads(line.strip()) for line in f if line.strip()]

print(f"Số lượng record: {len(data)}")

Đọc dữ liệu từ: /kaggle/input/vlsp-dataset/public_test.txt
Số lượng record: 400


In [ ]:
# ====== Inference & Ghi file ======
def classify_option(context, question, option):
    prompt = f"""
Bạn là trợ lý AI, nhiệm vụ phân loại các phương án trả lời về thời lượng:
- Input là JSON có fields "context", "question", "options" (là list các string)
- Output chỉ trả về JSON list "labels" có cùng độ dài với "options", giá trị "yes" nếu option đúng mô tả thời lượng, ngược lại "no".
- Các phương án trả lời có thể là thời gian thực tế hoặc ước lượng, không cần chính xác tuyệt đối.
- Trong mỗi question, không được phép gán tất cả các labels là "yes" hoặc "no".

Bây giờ phân loại cho record sau:
{json.dumps(record, ensure_ascii=False)}
"""
    try:
        output = chat_pipeline(prompt)[0]["generated_text"]
        for line in output.split('\n')[::-1]:
            if "yes" in line.lower():
                return "yes"
            if "no" in line.lower():
                return "no"
    except Exception as e:
        print(f"[Lỗi mô hình] {e}")
    return "no"

print("Đang suy luận...")
output_data = []

for ex in tqdm(data):
    preds = []
    for opt in ex["options"]:
        pred = classify_option(ex["context"], ex["question"], opt)
        preds.append(pred)

    record_out = {
        "context": ex["context"],
        "question": ex["question"],
        "options": ex["options"],
        "qid": ex["qid"],
        "labels": ex.get("labels", []),  # fallback nếu không có
        "predictions": preds
    }

    output_data.append(record_out)


Đang suy luận...


 10%|█         | 40/400 [59:25<10:15:27, 102.58s/it]

In [ ]:
# ====== Save Output ======
OUTPUT_PATH = "/kaggle/working/vistral_predictions.txt"

print(f"Ghi kết quả ra: {OUTPUT_PATH}")
with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    for item in output_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print("Hoàn tất!")


In [ ]:
import os

# liệt kê tất cả file trong /kaggle/working/
print(os.listdir("/kaggle/working/"))
